# Subscriptions API Training
---

## Introduction
----
The Subscriptions API combines the capabilities of Planet Search, Processing, Delivery into a single API. With this API, users can subscribe to continuous delivery of the imagery and metadata, and imagery delivered as soon as its available.


This tutorial notebook is an introduction to the Subscriptions API. Using simple Python code, it demonstrates creating, editing, and canceling a subscription. 


More information can be found in [Subscriptions API Documentation](https://developers.planet.com/docs/subscriptions/) in Planet's Developer Resource Center.

## Table of Contents

1. Imports 
2. Authentication
3. Requests example
4. Create a subscription
5. Edit a subscription
6. Cancel a subscription
7. Get info on subscription
---

### 1. Import required modules
You'll need the following python packages to run this code:

In [60]:
from requests.auth import HTTPBasicAuth
import requests
import os
import json
import itertools
from datetime import datetime, timedelta, timezone
import time
import glob
import pytz

### 2. Authentication

Authentication with the Planet Subscription API be achieved using a valid Planet **API key**.

In [61]:
# API Key stored as an env variable or simply replace os.getenv('') by your API key
PLANET_API_KEY = os.getenv('PL_API_KEY')

In [62]:
subscriptions_url = "https://api.planet.com/subscriptions/v1"

#### Curl Example


To check your orders list and make sure you have the permissions you need, uncomment the following line to run ```curl```

In [63]:
#!curl -L -H "Authorization: api-key $PLANET_API_KEY" $subscriptions_url

### 3. Requests example

In this notebook, we will be using `requests` to communicate with the Subscriptions API. First, we will check our orders list to make sure authentication and communication is working as expected.

We want to get a response code of `200` from this API call. To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/docs/orders/reference/#operation/listOrders) AOI reference.

If you recieve a `401` make sure that you correclty entered your API key

In [64]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(subscriptions_url, auth=auth)
response

<Response [200]>

Now we will list the subscriptions we have created thus far. Your list may be empty if you have not created any subscriptions yet.


In [65]:
subscriptions = response.json()['subscriptions']
len(subscriptions)

8

### 4. Creating a Subscription


This example creates a subscription that will deliver imagery between March and October for 2021, 2022, and 2023 over Tokyo.

In this subscription we request a PSScene of the ortho_analytic_4b asset type. The subscription delivers to google cloud bucket, but other cloud delivery options are explained [Here](https://developers.planet.com/docs/subscriptions/delivery/)

In [66]:
# set content type to json
headers = {'content-type': 'application/json'}

# set your delivery details
BUCKET_NAME = 'name of bucket'
GCS_CREDENTIALS= "credentials string"

#### Creating Request

In [67]:
request = {
    "name": "Recurring Subscription",
    "source": {
        "type": "catalog",
        "parameters": {
            "geometry": {
                "coordinates": [[[139.5648193359375,35.42374884923695],
                                [140.1031494140625,35.42374884923695],
                                [140.1031494140625,35.77102915686019],
                                [139.5648193359375,35.77102915686019],
                                [139.5648193359375,35.42374884923695]]],
                "type": "Polygon"
            },
            "start_time": "2022-03-01T00:00:00Z",
            "end_time": "2023-11-01T00:00:00Z",
            "rrule": "FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10",
            "item_types": ["PSScene"],
            "asset_types": ["ortho_analytic_4b"]
        }
    },
    "delivery": { 
        "type": "google_cloud_storage",
        "parameters": {
            "bucket": BUCKET_NAME,
            "credentials": GCS_CREDENTIALS,
                
        }
    }
}

In [68]:
def place_subscription(request, auth):
    response = requests.post(subscriptions_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    subscriptions_id = response.json()['id']
    print(subscriptions_id)
    subscription_url = subscriptions_url + '/' + subscriptions_id
    return subscription_url

If you are getting errors due to your request try replacing print(response) with print(response.json()) to get a better idea of the issue.

In [69]:
subscription_url = place_subscription(request, auth)

<Response [200]>
284f35d8-1723-4d14-8270-81d7773939b0


### 5. Edit a Subscription

When a subscription is in `pending` state, it may be edited in full. After the subscription transitions to `running`, the following source block edits will be disallowed:

  * Changes to `item_types`
  * Changes to `start_time` values

If you want to use access a different subscription you can use step <b>7. Get Info on Subscription</b> to print the ID's and paste those into the url

In [70]:
# report order state
#subscription_url = "https://api.planet.com/subscriptions/v1/SUB_ID"
requests.get(subscription_url, auth=auth).json()['status']

'preparing'

Check the perameters that you wish to edit, in this case the `"end_time"`

In [71]:
print(request["source"]["parameters"]["end_time"])
with open('data.json', 'w') as f:
    json.dump(request, f)

2023-11-01T00:00:00Z


In this case lets edit the `"end_time"` of our request to 2024

In [72]:
# PUT https://api.planet.com/subscriptions/v1/<subscription_id>
request["source"]["parameters"]["end_time"] = '2024-03-01T00:00:00Z'
response = requests.put(subscription_url, json=request, auth=auth)
response

<Response [200]>

In [78]:
response.json()["source"]["parameters"]["end_time"]

'2024-03-01T00:00:00Z'

### 6. Cancel a Subscription

If you want to use access a different subscription you can use step <b>7. Get Info on Subscription</b> to print the ID's and paste those into the url

In [79]:
# report order state
#subscription_url = "https://api.planet.com/subscriptions/v1/SUB_ID"
print(subscription_url)
requests.get(subscription_url, auth=auth).json()['status']

https://api.planet.com/subscriptions/v1/284f35d8-1723-4d14-8270-81d7773939b0


'pending'

In [80]:
# POST https://api.planet.com/subscriptions/v1/<subscription_id>/cancel
cancel_url = subscription_url + "/cancel"
response = requests.post(cancel_url, auth=auth)
response


<Response [200]>

In [81]:
# report order state - it could take a little while to cancel
requests.get(subscription_url, auth=auth).json()['status']

'cancelled'

### 7. Get Info on Subscriptions

By placing a get command you will print all of your subscriptions. 

In [84]:
response = requests.get(subscriptions_url, auth=auth)
response.json()

You can then easily itterate through them in order to get the `'id'`, `'status'`, or `'end_time'`

In [85]:
response = requests.get(subscriptions_url, auth=auth)
for i in range(len(response.json()['subscriptions'])):
    print(response.json()['subscriptions'][i]['id'])
    print(response.json()['subscriptions'][i]['status'])
    print(response.json()['subscriptions'][i]['source']['parameters']['end_time'])